# Compute the prior probability of treatment using permutation 

In [17]:
import itertools
import statistics
from tqdm import tqdm

import pandas

from hetio.permute import permute_pair_list

In [10]:
# Read treatments
treatment_df = pandas.read_table('../summary/indications.tsv')
treatment_df = treatment_df.query("rel_type == 'drug-used-for-treatment_DduftC'")
treatment_df.head(2)

,compound_id,compound_name,disease_id,disease_name,rel_type
0,Q413147,(+)-phenylpropanolamine,Q12125,Common cold,drug-used-for-treatment_DduftC
1,Q413147,(+)-phenylpropanolamine,Q12174,Obesity,drug-used-for-treatment_DduftC


In [11]:
# Create node to degree dictionaries
compound_to_degree = dict(treatment_df.compound_id.value_counts())
disease_to_degree = dict(treatment_df.disease_id.value_counts())

In [12]:
# A degree (compound_degree, disease_degree) to all potential edges with that degree
degree_to_edges = dict()

rows = list()
for (c, c_deg), (d, d_deg) in itertools.product(compound_to_degree.items(), disease_to_degree.items()):
    rows.append((c, d, c_deg, d_deg))
    degree = c_deg, d_deg
    edge = c, d
    degree_to_edges.setdefault(degree, set()).add(edge)

pair_df = pandas.DataFrame(rows, columns=['compound_id', 'disease_id', 'compound_treats', 'disease_treats'])
pair_df = pair_df.sort_values(['compound_id', 'disease_id'])

In [13]:
treatments = list(zip(treatment_df.compound_id, treatment_df.disease_id))

In [14]:
# Burn In
pair_list, stats = permute_pair_list(treatments, multiplier=10)
pandas.DataFrame(stats)

,attempts,complete,cumulative_attempts,duplicate,excluded,same_edge,self_loop,unchanged,undirected_duplicate
0,2970,0.100034,2969,0.043771,0,0.001010,0,0.160997,0
1,2969,0.200034,5938,0.054564,0,0.000337,0,0.048501,0
2,2969,0.300034,8907,0.044123,0,0.000674,0,0.031660,0
3,2969,0.400034,11876,0.047828,0,0.000337,0,0.023914,0
4,2969,0.500034,14845,0.047491,0,0.000000,0,0.027282,0
5,2969,0.600034,17814,0.048164,0,0.000337,0,0.024251,0
6,2969,0.700034,20783,0.045133,0,0.000000,0,0.032334,0
7,2969,0.800034,23752,0.046143,0,0.000337,0,0.026945,0
8,2969,0.900034,26721,0.051869,0,0.000674,0,0.032671,0
9,2968,1.000000,29689,0.050539,0,0.001011,0,0.028292,0


In [15]:
# Set the multiplier based on the burn in stats
multiplier = 3

In [19]:
# Calculate the number of perms
n_perm = treatment_df.compound_id.nunique() * treatment_df.disease_id.nunique()
#n_perm = int(n_perm * 25)
n_perm

637894

In [20]:
%%time

# Initialize a dictionary of degree to empirical probability list
degree_to_probs = {x: list() for x in degree_to_edges}

# Perform n_perm permutations
for i in tqdm(range(n_perm)):
    # Permute
    pair_list, stats = permute_pair_list(pair_list, multiplier=multiplier, seed=i)
    
    # Update
    pair_set = set(pair_list)
    for degree, probs in degree_to_probs.items():
        edges = degree_to_edges[degree]
        probs.append(len(edges & pair_set) / len(edges))


100%|██████████| 637894/637894 [15:05:18<00:00, 11.29it/s]

CPU times: user 15h 3min 23s, sys: 1min 26s, total: 15h 4min 49s
Wall time: 15h 5min 18s


In [25]:
%%time
rows = list()
for (c_deg, d_deg), probs in degree_to_probs.items():
    mean = statistics.mean(probs)
    std_error = statistics.stdev(probs) / len(probs) ** 0.5
    rows.append((c_deg, d_deg, mean, std_error))
perm_df = pandas.DataFrame(rows, columns=['compound_treats', 'disease_treats', 'prior_perm', 'prior_perm_stderr'])
perm_df = perm_df.sort_values(['compound_treats', 'disease_treats'])

CPU times: user 59min 23s, sys: 15.7 s, total: 59min 39s
Wall time: 59min 39s


In [26]:
# Add unpermuted treatment prevalence columns
rows = list()
treatment_set = set(treatments)
for (c_deg, d_deg), edges in degree_to_edges.items():
    n_treatments = len(edges & treatment_set)
    rows.append((c_deg, d_deg, n_treatments, len(edges)))
degree_prior_df = pandas.DataFrame(rows, columns=['compound_treats', 'disease_treats', 'n_treatments', 'n_possible'])
degree_prior_df = perm_df.merge(degree_prior_df)
degree_prior_df = degree_prior_df.sort_values(['compound_treats', 'disease_treats'])

In [27]:
degree_prior_df.tail(2)

,compound_treats,disease_treats,prior_perm,prior_perm_stderr,n_treatments,n_possible
406,19,51,0.733662,0.000512,0,1
407,19,68,0.795997,0.000467,1,1


In [28]:
degree_prior_df.to_csv('data/degree-prior.tsv', sep='\t', index=False, float_format='%.6g')

In [29]:
obs_prior_df = pair_df.merge(perm_df)

In [30]:
obs_prior_df.head(2)

,compound_id,disease_id,compound_treats,disease_treats,prior_perm,prior_perm_stderr
0,DB00014,DOID:0050741,2,4,0.009801,0.000005
1,DB00014,DOID:10652,2,4,0.009801,0.000005


In [31]:
len(obs_prior_df)

29799

In [32]:
obs_prior_df.to_csv('data/observation-prior.tsv', sep='\t', index=False, float_format='%.6g')